In [5]:
# Import Libraries for my AI Agent
import os
from dotenv import load_dotenv
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools
from phi.tools.duckduckgo import DuckDuckGo

In [6]:
# Load environment variables
load_dotenv()

# Get API keys from environment variables
PHI_API_KEY = os.getenv("PHI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [7]:
# Verify API keys
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY not found in environment variables")

# Initialize Groq LLM with error handling
try:
    groq_llm = Groq(
        id="llama3-70b-8192",
        api_key=GROQ_API_KEY,
        temperature=0.3  # Lower temperature for more factual responses
    )
except Exception as e:
    raise RuntimeError(f"Failed to initialize Groq client: {str(e)}")

In [8]:
# Create agents with error handling
try:
    # Web Search Agent
    web_search_agent = Agent(
        name="Web Search Agent",
        role="Searches the web for current information",
        model=groq_llm,
        tools=[DuckDuckGo()],
        introduction="Always verify information and include sources.",
        show_tool_calls=True
    )

    # Financial Agent
    financial_agent = Agent(
        name="Financial Analyst",
        role="Analyzes stock market data and company fundamentals",
        model=groq_llm,
        tools=[
            YFinanceTools(
                stock_price=True,
                analyst_recommendations=True,
                stock_fundamentals=True,
                company_news=True
            )
        ],
        introduction="Present financial data clearly with tables when appropriate.",
        show_tool_calls=True
    )

    # Multi-Agent System
    multi_agent = Agent(
        name="Research Team",
        role="Coordinates research between web search and financial analysis",
        team=[web_search_agent, financial_agent],
        model=groq_llm,
        instructions=[
            "First gather current news using the web search agent",
            "Then analyze financial data using the financial agent",
            "Combine both sources for a comprehensive report"
        ],
        show_tool_calls=True
    )
except Exception as e:
    raise RuntimeError(f"Failed to create agents: {str(e)}")

# Execute query with error handling
try:
    query = "Summarize and analyze the latest news about Apple Inc. (AAPL) stock."
    print(f"Executing query: {query}")

    # First try with streaming
    try:
        print("\nStreaming response:")
        for chunk in multi_agent.run(query, stream=True):
            if chunk and hasattr(chunk, 'content'):
                print(chunk.content, end="", flush=True)
    except Exception as stream_error:
        print(f"\nStreaming failed, falling back to regular response: {str(stream_error)}")
        full_response = multi_agent.run(query)
        print("\nFull response:")
        print(full_response)

except Exception as e:
    print(f"Error executing query: {str(e)}")
    print("Possible solutions:")
    print("1. Verify your GROQ_API_KEY is valid")
    print("2. Check your internet connection")
    print("3. Try simplifying your query")
    print("4. Wait a few minutes and try again (API might be busy)")

Executing query: Summarize and analyze the latest news about Apple Inc. (AAPL) stock.

Streaming response:

Running:
 - transfer_task_to_web_search_agent(task_description=Provide the latest news about Apple Inc. (AAPL) stock, expected_output=A list of recent news articles about Apple Inc. (AAPL) stock, additional_information=Focus on reputable sources and provide a brief summary of each article)
 - transfer_task_to_financial_analyst(task_description=Analyze the current financial data of Apple Inc. (AAPL) stock, expected_output=A report on the current stock price, company fundamentals, and analyst recommendations, additional_information=Provide a comprehensive analysis of the company's financial health)


Running:
 - transfer_task_to_web_search_agent(task_description=..., expected_output=A comprehensive report combining the latest news and financial analysis of Apple Inc. (AAPL) stock, additional_information=Focus on providing a clear and concise summary of the company's current situati